## Yolo11 tests

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os , sys
import shutil
from tqdm.auto import tqdm  
sys.path.append('..')
from pathlib import Path
cur_path = Path(os.getcwd()).parent
sam2_path = cur_path / 'sam2_octron'
sys.path.append(cur_path.as_posix())
from matplotlib import pyplot as plt
import cmasher as cmr
import numpy as np
import seaborn as sns
sns.set_theme(style='white')
%config InlineBackend.figure_format = 'retina'

In [3]:
from napari_pyav._reader import FastVideoReader
from octron.sam2_octron.helpers.video_loader import get_vfile_hash
from octron.yolo_octron.helpers.training import (collect_labels, 
                                                 collect_polygons,
                                                 draw_polygons,
                                                 train_test_val,
                                                 write_training_data,
                                                 write_yolo_config_yaml,
                                                 
)
from octron.yolo_octron.helpers.polygons import (find_objects_in_mask, 
                                                 watershed_mask,
                                                 get_polygons,
)
from octron.sam2_octron.helpers.sam2_zarr import load_image_zarr

In [4]:
# Presaved model (YOLOv11)
path_to_model = Path('/Users/horst/Documents/python/OCTRON/octron/yolo_octron/yolo11l-seg.pt')
project_path = Path('/Users/horst/Downloads/octron_project_2')
assert project_path.exists()
assert path_to_model.exists()

### Find out what info is present across object organizers / ...

- Construct a nice loop that finds the zarr, video files, and compares video file hashes 


In [5]:
project_path

PosixPath('/Users/horst/Downloads/octron_project_2')

In [6]:
label_dict = collect_labels(project_path, prune_empty_labels=True, min_num_frames=10, verbose=False)
label_dict = collect_polygons(label_dict)  

📖 Octron object organizer loaded from /Users/horst/Downloads/octron_project_2/f080acc6/object_organizer.json
📖 Octron object organizer loaded from /Users/horst/Downloads/octron_project_2/7c724296/object_organizer.json
📖 Octron object organizer loaded from /Users/horst/Downloads/octron_project_2/c22c039f/object_organizer.json


Polygons for label worm:   0%|          | 0/99 [00:00<?, ?it/s]

Polygons for label worm:   0%|          | 0/57 [00:00<?, ?it/s]

Polygons for label led:   0%|          | 0/57 [00:00<?, ?it/s]

Polygons for label worm:   0%|          | 0/58 [00:00<?, ?it/s]

Polygons for label led:   0%|          | 0/58 [00:00<?, ?it/s]

In [7]:
# for folder, l in label_dict.items():
#     print(folder)
#     draw_polygons(l, l['video'], 
#                   max_to_plot=1,
#                   randomize=True
#                  )   
    

In [8]:
# Perform the split of the data
# And save frames back into 'frames_split' key
print('Splitting the data for training into training, validation and testing fractions')  
for labels in label_dict.values():
    for entry in labels:
        if entry == 'video':
            continue    
        label = labels[entry]['label']
        print(f'Processing {label} ...')
        frames = labels[entry]['frames']   
        split_dict = train_test_val(frames, 
                                     training_fraction=0.7,
                                     validation_fraction=0.15,
                                     verbose=True
                                     )

        labels[entry]['frames_split'] = split_dict

Splitting the data for training into training, validation and testing fractions
Processing worm ...
Total frames: 99
Training set: 69 frames
Validation set: 14 frames
Test set: 16 frames
Processing worm ...
Total frames: 57
Training set: 39 frames
Validation set: 8 frames
Test set: 10 frames
Processing led ...
Total frames: 57
Training set: 39 frames
Validation set: 8 frames
Test set: 10 frames
Processing worm ...
Total frames: 58
Training set: 40 frames
Validation set: 8 frames
Test set: 10 frames
Processing led ...
Total frames: 58
Training set: 40 frames
Validation set: 8 frames
Test set: 10 frames


In [9]:
path_to_training = project_path / 'model'
try:
    path_to_training.mkdir(exist_ok=False)
except FileExistsError as e:
    # Check if path_to_training_data is empty
    if len(list(path_to_training.glob('*'))) > 1:
       raise FileExistsError(f'{path_to_training} is not empty. Please remove subfolders first.')
    else:
        pass

path_to_training_data = project_path / 'model' / 'training_data'
write_training_data(label_dict, path_to_training_data, verbose=True)

dataset_path = path_to_training_data
train_path = "train"  
val_path = "val"
test_path = "test"

# Get label names from the object organizer
label_id_label_dict = {}
for label_id, label_dict in labels.items():
    label_id_label_dict[label_id] = label_dict['label']

# Write the YAML config
config_path = path_to_training / "yolo_config.yaml"
_ = write_yolo_config_yaml(
    output_path=config_path,
    dataset_path=dataset_path,
    train_path=train_path,
    val_path=val_path,
    test_path=test_path,
    label_dict=label_id_label_dict
)

Exporting 1 labels:   0%|          | 0/1 [00:00<?, ?it/s]

Exporting train frames:   0%|          | 0/69 [00:00<?, ?it/s]

Exporting val frames:   0%|          | 0/14 [00:00<?, ?it/s]

Exporting test frames:   0%|          | 0/16 [00:00<?, ?it/s]

Exporting 2 labels:   0%|          | 0/2 [00:00<?, ?it/s]

...

Exporting test frames:   0%|          | 0/10 [00:00<?, ?it/s]

Exporting train frames:   0%|          | 0/40 [00:00<?, ?it/s]

Exporting val frames:   0%|          | 0/8 [00:00<?, ?it/s]

Exporting test frames:   0%|          | 0/10 [00:00<?, ?it/s]

Training data exported to /Users/horst/Downloads/octron_project_2/model/training_data
YOLO config saved to /Users/horst/Downloads/octron_project_2/model/yolo_config.yaml


## YOLO setup

In [10]:
from ultralytics import settings
settings.update({'sync':False,'hub':False,'runs_dir':path_to_training.as_posix()})
from ultralytics import YOLO

In [ ]:
# Load a model
model = YOLO(path_to_model)  # load a pretrained model (recommended for training)
# Train the model
# https://docs.ultralytics.com/usage/cfg/#solutions-settings
results = model.train(data=config_path, 
                      save_dir=path_to_training.as_posix(),
                      device='cpu',
                      mask_ratio=4,
                      epochs=60,
                      imgsz=640,
                      resume=False,
                      plots=True,
                      batch=.9,
                      cache=False,
                      save=True,
                      save_period=15,
                      project=None,
                      name=None,
                      exist_ok=True,
                      # augmentation
                      hsv_v=.25,
                      degrees=180,
                      scale=.5,
                      shear=2,
                      flipud=.1,
                      fliplr=.1,
                      mosaic=1.0,
                      copy_paste=.5,
                      copy_paste_mode='mixup', 
                      erasing=.25,
                      crop_fraction=1.0,
                      )

New https://pypi.org/project/ultralytics/8.3.82 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.81 🚀 Python-3.11.11 torch-2.3.1 CPU (Apple M2 Max)
engine/trainer: task=segment, mode=train, model=/Users/horst/Documents/python/OCTRON/octron/yolo_octron/yolo11l-seg.pt, data=/Users/horst/Downloads/octron_project_2/model/yolo_config.yaml, epochs=60, time=None, patience=100, batch=0.9, imgsz=640, save=True, save_period=15, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_

train: Scanning /Users/horst/Downloads/octron_project_2/model/training_data/train... 148 images, 0 backgrounds, 0 corrupt: 100%|██████████| 148/148 [00:00<00:00, 1892.20it/s]

train: New cache created: /Users/horst/Downloads/octron_project_2/model/training_data/train.cache
AutoBatch: Computing optimal batch size for imgsz=640 at 90.0% CUDA memory utilization.
AutoBatch:  ⚠️ intended for CUDA devices, using default batch-size 16



train: Scanning /Users/horst/Downloads/octron_project_2/model/training_data/train.cache... 148 images, 0 backgrounds, 0 corrupt: 100%|██████████| 148/148 [00:00<?, ?it/s]
val: Scanning /Users/horst/Downloads/octron_project_2/model/training_data/val... 30 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30/30 [00:00<00:00, 978.76it/s]

val: New cache created: /Users/horst/Downloads/octron_project_2/model/training_data/val.cache


Plotting labels to /Users/horst/Downloads/octron_project_2/model/segment/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 176 weight(decay=0.0), 187 weight(decay=0.0005), 186 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/horst/Downloads/octron_project_2/model/segment/train
Starting training for 60 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/60         0G     0.8698      1.417      3.291     0.9354         68        640:  50%|█████     | 5/10 [03:01<03:01, 36.35s/it]

In [ ]:
# model = YOLO('/Users/horst/Downloads/octron_project/octron_training/yolo runs/segment/train/weights/last.pt')  #

In [ ]:
# metrics = model.val(device='cpu', plots=True)

In [ ]:
# print("Mean Average Precision for boxes:", metrics.box.map)
# print("Mean Average Precision for masks:", metrics.seg.map)

In [ ]:
# # Run inference on 'bus.jpg' with arguments
# model.predict('/Users/horst/Downloads/octron_project/test data/8_behaviour_filtered2024-11-04T14_20_34_20240930_Th19.mp4', 
#               save=True, 
#               classes=[0],
#               imgsz=1000, 
#               device='cpu',
#               visualize=False,
#               conf=0.9
#               )

In [ ]:
# # Train/val/test sets as 
# 1) dir: path/to/imgs, 
# 2) file: path/to/imgs.txt, or list: [path/to/imgs1, path/to/imgs2, ..]
# path: ../datasets/coco8-seg # dataset root dir (absolute or relative; if relative, it's relative to default datasets_dir)
# train: images/train # train images (relative to 'path') 4 images
# val: images/val # val images (relative to 'path') 4 images
# test: # test images (optional)
